In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
from numpy import asarray
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split, KFold,GridSearchCV
from sklearn.metrics import mean_squared_error,cohen_kappa_score, make_scorer

# xgboost
!pip install xgboost
import xgboost as xgb
from xgboost import XGBRegressor

In [ ]:
#load dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/Data/training_set_rel3.tsv",sep='\t', encoding='ISO-8859-1')
dev = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/Data/valid_set.tsv",sep='\t', encoding='ISO-8859-1')
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/Data/test_set.tsv",sep='\t', encoding='ISO-8859-1')

In [ ]:
## import feature

# for semantic score
semantic_results_lstm = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/Data/semantic_score_train_lstm.csv")
semantic_results_bert = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/Data/semantic_results_train_bert.csv")

# for coherence score
local_coherence_results = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/Data/local_coherence_results_train.csv")
global_coherence_results = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/Data/global_coherence_results_train.csv")

# for prompt relevant score
cosine_similarity = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/Data/Cosine_similarity.csv")
prompt_relevant_results = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/Data/prompt_results_train.csv")

# handcrafted features
handcrafted_features = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/Data/handcrafted_features_train.csv")

In [ ]:
local_coherence_results = local_coherence_results.groupby(["essay_id"]).agg({"local_coherence_results":"mean"}).reset_index()

In [ ]:
final = pd.concat([train[['essay_id','essay_set','domain1_score']]
                   , semantic_results_lstm[['semantic_results_lstm']]
                   , cosine_similarity[['cosine_similarity']]
                   , handcrafted_features[['incorrect_spelling','correct_spelling','lexical_diversity','noun_cnt','verb_cnt','adv_cnt','adj_cnt']]]
                  ,axis=1)

In [ ]:
final = final.merge(
    prompt_relevant_results,how='inner',on='essay_id'
    ).merge(
        global_coherence_results,how='inner',on='essay_id'
        ).merge(semantic_results_bert,on='essay_id',how='inner').merge(local_coherence_results,how='left',on='essay_id')

In [ ]:
final.loc[final['local_coherence_results'].isna(),'local_coherence_results']=0

In [ ]:
final.head()

,essay_id,essay_set,domain1_score,semantic_results_lstm,cosine_similarity,incorrect_spelling,correct_spelling,lexical_diversity,noun_cnt,verb_cnt,adv_cnt,adj_cnt,prompt_relevant_score,global_coherence_results,semantic_results_bert,local_coherence_results
0,1,1,8,0.689008,0.945513,36,286,2.19,75,57,21,19,0.844146,0.712342,0.691455,0.682910
1,2,1,9,0.745791,0.957986,24,361,2.21,95,73,25,22,0.838242,0.771744,0.731046,0.734613
2,3,1,7,0.752003,0.981550,9,251,2.01,71,45,14,19,0.692520,0.636964,0.615243,0.706937
3,4,1,10,0.761740,0.960965,40,435,2.22,131,76,26,42,0.859761,0.796701,0.791721,0.717029
4,5,1,8,0.727743,0.962343,24,421,2.52,107,73,39,29,0.841044,0.857217,0.762809,0.653226


## xgboost 

In [ ]:
def xgboost(X,y, gamma = 1, learning_rate = 0.01, max_depth = 5, n_estimators = 5, subsample = 0.8, random_state=34):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=123)
  # define the model for multi-class classification
  mod = xgb.XGBClassifier(
    objective="multi:softmax",
    gamma=gamma,                 
    learning_rate=learning_rate,
    max_depth=max_depth,
    n_estimators=n_estimators,                                                                    
    subsample=subsample,
    random_state=random_state) 
  
  mod.fit(X_train, y_train)
  preds = mod.predict(X_test)
  QWK=cohen_kappa_score(y_test.astype(int), preds.astype(int), labels=None, weights= 'quadratic', sample_weight=None)
  return QWK